In [ ]:
!pip install google.colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


In [ ]:
!nvidia-smi

Fri Feb  7 07:15:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%cd /content/drive/My Drive/bert

/content/drive/My Drive/bert


In [ ]:
!pip install torch torchvision torchaudio transformers datasets evaluate accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

True
0


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

import random
from sklearn.metrics import f1_score

In [ ]:
'''
函式
'''
# 讀取 .txt 文件
def load_dataset_from_file(file_path, seed=42):
    # 讀檔
    with open(file_path, "r", encoding='utf-8') as file:
        # 將每一行資料以 list 型態回傳
        lines = file.readlines()

        # 跳過第一行（標題行），不然會因為無法轉成數字導致後面錯誤
        lines = lines[1:]

        # 洗牌 (記得設定 random seed，確保每次洗牌結果一樣)
        random.seed(seed)
        random.shuffle(lines)

        # 整合訓練資料
        sentences = []
        labels = []

        # 定義語調到數字的映射字典
        tone_map = {
            '平淡語氣': 0,
            '開心語調': 1,
            '悲傷語調': 2,
            '憤怒語調': 3,
            '驚奇語調': 4,
            '厭惡語調': 5,
            '疑問語調': 6,
            '關切語調': 7
        }

        # 逐行讀取資料
        for line in lines:
            # 每一行資料的"用從後面數過來第一個逗號"作為分隔符號
            parts = line.strip().rsplit(',', 1)

            # 確保每一行資料都有兩個部分
            if len(parts) == 2:
                sentences.append(parts[0])

                # 轉換語調字符串為數字
                tone = parts[1]
                if tone in tone_map:
                    labels.append(tone_map[tone])
                else:
                    print(f'未知語調: {tone}')
            else:
                print(f'格式錯誤的行號: {line}')

        return sentences, labels

# 轉換成 huggingface trainer 可以使用的 datasets
def convert_to_dataset(sentences, labels, tokenizer, max_seq_length):
    # 使用 tokenizer 將句子轉換為 tokenized 形式
    encodings = tokenizer(
        sentences,
        truncation=True,  # 長度超過 max_seq_length 的句子會被截斷
        padding='max_length',  # 將每個句子填充到 max_seq_length 長度
        max_length=max_seq_length,  # 設定最大長度
        return_tensors="pt"  # 轉換成 PyTorch tensor 格式
    )

    # 包裝成 dataset 格式，將 labels 加入
    encodings['labels'] = torch.tensor(labels)

    # 建立 Dataset 物件
    dataset = Dataset.from_dict(encodings)

    # 回傳切分資料 (訓練 和 驗證)
    dataset = dataset.train_test_split(test_size=0.2)

    return dataset
    '''
    print(dataset) 的內容如下:

    DatasetDict({
        train: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 6212
        })
        test: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 1554
        })
    })
    '''

    # 預處理資料
    def preprocess_data(dataset):
        # 將句子轉換為 token (tokenization)
        return tokenizer(
            dataset['sentences'],
            truncation=True,
            padding=True,
            return_tensors='pt',
            max_length=max_seq_length
        )

    # 轉換資料
    train_data = dataset['train'].map(preprocess_data, batched=True)
    valid_data = dataset['test'].map(preprocess_data, batched=True)

    return DatasetDict({
        'train': train_data,
        'test': valid_data
    })

# 計算模型評估指標
def compute_metrics(predicted_results):
    labels = predicted_results.label_ids
    preds = predicted_results.predictions.argmax(-1)
    '''
    為什麼是 argmax(-1)？

    import numpy as np
    predictions = np.array([
        [0.1, 0.9],
        [0.8, 0.2],
        [0.4, 0.6],
    ])
    preds = predictions.argmax(axis=-1)
    print(preds)

    np.argmax() 會找出最大的值，並回傳索引值
    - 如果是 axis=0，則會是 [1, 0] (0.8 和 0.9)，因為由上而下，對每一個欄位取最大值
    - 如果是 axis=1，則會是 [1, 0, 1] (0.9、0.8 和 0.6)，因為由左而右，對每一個列取最大值
    - 如果是 axis=-1，則會以最後一個維度為基礎，取最大值
    '''

    # 計算 F1 score (二元分類選 binary，多元分類可選 micro, macro, weighted)
    f1 = f1_score(labels, preds, average='micro')
    return {
        'f1': f1,
    }

In [ ]:
#抓出8個語調
file_path = "./data.csv"
with open(file_path, "r", encoding='utf-8') as file:
    # 將每一行資料以 list 型態回傳
    lines = file.readlines()


    # 整合訓練資料
    sentences = []
    labels = []

    # 逐行讀取資料
    for line in lines:
        # 每一行資料的"用從後面數過來第一個逗號"作為分隔符號
        parts = line.strip().rsplit(',', 1)

        # 確保每一行資料都有兩個部分
        if len(parts) == 2:
            sentences.append(parts[0])
            labels.append(parts[1])
        else:
            print(f'格式錯誤的行號: {line}')


    # 去除 labels 中的重複並保持順序
    labels = list(dict.fromkeys(labels))  # 使用 dict.fromkeys 去重並保持順序

    print(labels)  # 顯示處理過的 labels 列表

['emotion', '平淡語氣', '開心語調', '悲傷語調', '憤怒語調', '驚奇語調', '厭惡語調', '疑問語調', '關切語調']


In [ ]:
from sklearn.metrics import f1_score

# 驗證 F1 score 算法
y_true = [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4]
y_pred = [0, 1, 2, 3, 4, 5, 6, 7, 1, 0, 2, 3, 4]

print(f1_score(y_true, y_pred, average='micro')) # binary, micro, macro

# 參考: https://blog.csdn.net/qq_40671063/article/details/130447922

0.8461538461538461


In [ ]:
# 主程式 - 微調模型
if __name__ == "__main__":
    '''
    設定 hyperparameters
    '''
    model_name = 'google-bert/bert-base-chinese' # 預訓練模型名稱
    max_seq_length = 512 # 可訓練的序列最大長度
    num_labels = 8 # 8元分類
    output_dir = './output' # 輸出模型資料夾

    # 讀取訓練資料
    sentences, labels = load_dataset_from_file('./data.csv')

    # 載入 tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 將資料轉換為 huggingface 可以使用的格式
    dataset = convert_to_dataset(
        sentences,
        labels,
        tokenizer,
        max_seq_length
    )

    # 讀取模型
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels
    )

    # 設定訓練參數
    training_args = TrainingArguments(
        output_dir=output_dir, # 輸出資料夾
        overwrite_output_dir=True,
        num_train_epochs=3, # 訓練回合數
        per_device_train_batch_size=32, # 批次大小
        per_device_eval_batch_size=32, # 批次大小
        gradient_accumulation_steps=2,
        learning_rate=0.00005, # 學習率 5e-5
        warmup_steps=100,
        weight_decay=0.01,
        eval_strategy="steps", # epoch, steps, no
        eval_steps=100,
        save_strategy="steps", # epoch, steps, no
        save_steps=100,
        save_total_limit=2,
        load_best_model_at_end=True,
        seed=42, # 隨機種子
        lr_scheduler_type="linear", # https://blog.csdn.net/muyao987/article/details/139319466
        report_to='wandb', # https://wandb.ai/
    )

    # 設定 Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        data_collator=None, # DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics,
    )

    # 開始訓練
    trainer.train()

    # 儲存模型
    trainer.save_model(output_dir) # , safe_serialization=True

    # 儲存 tokenizer
    tokenizer.save_pretrained(output_dir)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1
100,No log,0.490442,0.856971


In [30]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
from pprint import pprint

output_dir = './output'
model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

list_text = [
    '你要不要去吃午餐',
    '誒誒誒！我甄選上了！',
    '我剛剛看到有人的眉毛像是蠟筆小新!',
]
result = pipe(list_text)

# 定義映射字典，將 LABEL_0, LABEL_1 映射回你的語調標籤
label_map = {
    0: '平淡語調',
    1: '開心語調',
    2: '悲傷語調',
    3: '憤怒語調',
    4: '驚奇語調',
    5: '厭惡語調',
    6: '疑問語調',
    7: '關切語調'
}

# 顯示預測結果並將其轉換為語調名稱
for text, item in zip(list_text, result):
    label_id = int(item['label'].split('_')[-1])  # 提取數字部分（LABEL_0 -> 0）
    predicted_label = label_map.get(label_id, '未知語調')
    print(f"'{text}' => {predicted_label} ({item['score']:.2f})")

Device set to use cuda:0


'你要不要去吃午餐' => 平淡語調 (0.88)
'誒誒誒！我甄選上了！' => 開心語調 (0.87)
'我剛剛看到有人的眉毛像是蠟筆小新!' => 驚奇語調 (0.58)


In [ ]:
pip list


Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.3.0
aiohappyeyeballs                   2.4.6
aiohttp                            3.11.12
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.4
ale-py                             0.10.1
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.6.0
arviz                              0.20.0
astropy                            7.0.1
astropy-iers-data                  0.2025.2.10.0.33.26
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              25.1.0
audioread          